In [ ]:
import sys
print(sys.version)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -l --block-size=K  '/content/drive/Shareddrives/KLUE Summarization/모델링/klue_sum_model_save'

total 576565K
-rw------- 1 root root      5K Oct 12 05:13 config.json
-rw------- 1 root root 575567K Oct 12 05:13 pytorch_model.bin
-rw------- 1 root root      1K Oct 12 05:13 special_tokens_map.json
-rw------- 1 root root      1K Oct 12 05:13 tokenizer_config.json
-rw------- 1 root root    656K Oct 12 05:13 tokenizer.json
-rw------- 1 root root    337K Oct 12 05:13 vocab.txt


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 636 kB 46.5 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizerFast, EncoderDecoderModel

In [ ]:
from os import truncate
# https://colab.research.google.com/github/kiyoungkim1/LMkor/blob/main/notebooks/summarization_with_bertshared.ipynb#scrollTo=atpZXK1ioMDa
# https://github.com/kiyoungkim1/LMkor/blob/main/examples/bertshared_summarization.py

class Summarize():
    def __init__(self, model_name):
        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        self.model = EncoderDecoderModel.from_pretrained(model_name)
        # self.model=self.model.encoder.resize_token_embeddings(len(self.tokenizer))
        # self.model=self.model.decoder.resize_token_embeddings(len(self.tokenizer))

    def __call__(self, text):
        input_ids = self.tokenizer.encode(text, return_tensors= 'pt',truncation=True,padding="max_length")

        sentence_length = len(input_ids[0])
        min_length = max(20, int(0.1*sentence_length))
        max_length = min(50, int(0.3*sentence_length))


        outputs = self.model.generate(
            input_ids,
            min_length=min_length,
            max_length=max_length
        )

        print(self.tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
summarize=Summarize('/content/drive/Shareddrives/KLUE Summarization/모델링/klue_sum_model_save')

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [ ]:
cd /content/drive/Shareddrives/KLUE Summarization/

/content/drive/Shareddrives/KLUE Summarization


In [ ]:
!ls

 모델링		 '문서요약 텍스트'   models		'Preprocessed Data'
'도서자료 요약'  'klue crawling'     news_summary2.pkl	'문서요약 텍스트_unzip'


In [ ]:
import json
import pandas as pd

def read_jsonl(path):
  DATA_DIR = "문서요약 텍스트_unzip"
  with open(DATA_DIR + path , 'r') as json_file:
      json_list = list(json_file)

  trains = []

  for json_str in json_list:
      line = json.loads(json_str)
      trains.append(line)
  df = pd.DataFrame(trains)
  return df

train_df = read_jsonl('/1.Training/신문기사_1.train.jsonl/train.jsonl')
val_df = read_jsonl('/2.Validation/신문기사_1.vaild.jsonl/vaild.jsonl')
train_df.head()

,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"


In [ ]:
def articles_to_str(articles_list):
  return ' '.join(articles_list)

In [ ]:
train_df['article_to_str']=train_df['article_original'].apply(articles_to_str)

In [ ]:
train_df.head()

,media,id,article_original,abstractive,extractive,article_to_str
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]",지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성...
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]",서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 우...
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]",지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대장...
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]",서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 2...
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]",미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민단...


In [ ]:
# Training data에 적용했을 때
text=train_df['article_to_str'][0]
print('본문 :',text)
print('------------------------------')
print('요약문 :',end='')
summarize(text)

본문 : 지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다. 폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계, 두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다. ‘연령표준화 사망률’(표준인구 10만 명당 사망자 수)은 울산·충북·부산 순으로 높게 나타났다. ■작년 사망자 29만 8820명, 역대 최다 24일 통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 전년 대비 4.7%(1만 3286명) 증가한 29만 8820명으로 관련 통계를 작성한 1983년 이후 가장 많았으며, 5년 연속 증가세를 보였다. 통계청은 인구 구조의 고령화와 지난해 1~2월 유례가 드문 한파 등을 그 원인으로 꼽았다. 지난해 조사망률(인구 10만 명당 사망자 수) 역시 582.5명으로 전년보다 4.5%(25.1명) 증가해 5년 연속 늘었다. 특히 80세 이상의 사망자가 전체 사망자의 절반에 가까운 46.3%로, 10년 전보다 14.3%포인트(P)나 증가했다. ■폐렴·알츠하이머병 사망률 순위 ‘껑충’ 사망원인별로 보면 지난해 암(악성신생물)에 의한 사망률(이하 인구 10만 명당 사망자 수)은 154.3명으로 전년보다 0.2% 증가했다. 1983년 관련 통계를 집계한 이래 줄곧 암이 사망원인 1위로 집계됐다. 특히 폐렴(4위→3위)과 치매의 일종인 알츠하이머병(11위→9위)에 의한 사망률 순위 상승이 두드러졌다. 폐렴 사망률은 2004년 10위에서 꾸준히 순위가 상승하고 있고, 알츠하이머병 사망률 역시 통계 작성 이래 10대 사인에 처음 포함됐다. 지난해 알츠하이머병에 의한 사망률은 12.0명으로 전년(9.8명) 대비 22.5% 증가했다. 알츠하이머병 사망률은 10년 전(3.8명)과 비교하면 무려 214.2% 증가했다. 폐렴 사망률은 45.4명으로 전년(37.8명) 대비 20.0% 증가했다. 알코올 관련 사망률은 9.6명으로 전년보다 2.0% 늘었다. ■

In [ ]:
text='이때까지 경영학과에서 들었던 모든 과목중에 성취감은 가장 높은 과목인 것 같습니다.매번 케이스 분석을 바탕으로 정말 오랜시간동안 토론이 진행되는데, 내용이 많이 유익합니다. 팀플이 까다로운 편이긴 하지만, 모두가 노력한 만큼 교수님께서 점수를 잘 주시는 것 같습니다. 쉽지 않은 과목이지만, 그만큼 얻어가는 것이 많았던 김재욱 교수님의 강의였습니다. 강의 시간이 길고, HBS 케이스 및 과제에 대한 부담이 컸지만, 별도의 시험을 치지 않기 때문에, 시험 기간의 부담이 훨씬 적었습니다. 교수님께서 이제 안식년에 들어가시기 때문에 당분간은 개강되지 않겠지만, 비즈니스 케이스에 대한 분석 방법론, 데이터 계산을 통한 근거 형성, 열정적인 강의 등에 관심있는 학우라면, 추천합니다! 줌 라이브 강의로 진행됨 / 월요일 3시반~6시15분(이지만 더 늦게 끝날 때도 있었어요) / 중간기말 없고, 1~2주에 한번 정도 영문 케이스 읽고 레포트(2장 분량) 제출 / 팀플 레포트+발표 1번 있음. 사실 이 강의를 들으면서 내용도 어렵고 케이스 스터디는 막막해서 힘들 때가 많았습니다. 산 넘어 산이라는 말처럼 매주 더 어려워지더라고요... 마케팅 잘 모르는 내가 이걸 왜 듣겠다고 했을까 후회도 했어요. 그런데 신기한 게 매주 끙끙대면서 레포트를 써서 제출하고 나서 수업을 들으면 신기하게 고민하던 내용들이 다 정리되는 느낌이었어요. 교수님의 설명+학우님들의 의견을 듣다 보면 명쾌하게 이해가 되는 기적... B2B 쪽은 개인적으로 처음 접해봐서 낯설고 어려웠지만 강의가 유익해서 열심히 들었던 것 같아요. 학점이 어떻게 나오더라도 후회 없이 최선을 다하자는 생각이었는데, 결과적으로는 기대 이상으로 학점도 잘 받았습니다. 수업 시간에 발표를 자주 하는 게 중요한 것 같아요. 레포트는 내가 어려우면 다른 사람도 어렵다 라는 생각으로 가능한 만큼 쓰면 (정신건강에) 좋을 것 같습니다ㅎㅎ 이 강의를 들으면서 B2B 마케팅과 더 친숙해졌고 실무적인 관점의 접근을 배운 것 같아요. 관련 분야에 관심 있는 분들께 추천드립니다.'

In [ ]:
print('본문 :',text)
print('------------------------------')
print('요약문 :',end='')
summarize(text)

본문 : 이때까지 경영학과에서 들었던 모든 과목중에 성취감은 가장 높은 과목인 것 같습니다.매번 케이스 분석을 바탕으로 정말 오랜시간동안 토론이 진행되는데, 내용이 많이 유익합니다. 팀플이 까다로운 편이긴 하지만, 모두가 노력한 만큼 교수님께서 점수를 잘 주시는 것 같습니다. 쉽지 않은 과목이지만, 그만큼 얻어가는 것이 많았던 김재욱 교수님의 강의였습니다. 강의 시간이 길고, HBS 케이스 및 과제에 대한 부담이 컸지만, 별도의 시험을 치지 않기 때문에, 시험 기간의 부담이 훨씬 적었습니다. 교수님께서 이제 안식년에 들어가시기 때문에 당분간은 개강되지 않겠지만, 비즈니스 케이스에 대한 분석 방법론, 데이터 계산을 통한 근거 형성, 열정적인 강의 등에 관심있는 학우라면, 추천합니다! 줌 라이브 강의로 진행됨 / 월요일 3시반~6시15분(이지만 더 늦게 끝날 때도 있었어요) / 중간기말 없고, 1~2주에 한번 정도 영문 케이스 읽고 레포트(2장 분량) 제출 / 팀플 레포트+발표 1번 있음. 사실 이 강의를 들으면서 내용도 어렵고 케이스 스터디는 막막해서 힘들 때가 많았습니다. 산 넘어 산이라는 말처럼 매주 더 어려워지더라고요... 마케팅 잘 모르는 내가 이걸 왜 듣겠다고 했을까 후회도 했어요. 그런데 신기한 게 매주 끙끙대면서 레포트를 써서 제출하고 나서 수업을 들으면 신기하게 고민하던 내용들이 다 정리되는 느낌이었어요. 교수님의 설명+학우님들의 의견을 듣다 보면 명쾌하게 이해가 되는 기적... B2B 쪽은 개인적으로 처음 접해봐서 낯설고 어려웠지만 강의가 유익해서 열심히 들었던 것 같아요. 학점이 어떻게 나오더라도 후회 없이 최선을 다하자는 생각이었는데, 결과적으로는 기대 이상으로 학점도 잘 받았습니다. 수업 시간에 발표를 자주 하는 게 중요한 것 같아요. 레포트는 내가 어려우면 다른 사람도 어렵다 라는 생각으로 가능한 만큼 쓰면 (정신건강에) 좋을 것 같습니다ㅎㅎ 이 강의를 들으면서 B2B 마케팅과 더 친숙해졌고 실무적인 관점의 접근을 배운 것 같아요. 관련 